In [1]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import sys

In [3]:
df = pd.read_csv('../../data/scraped_text/amazon_product_name.csv')
df.reset_index(inplace=True, drop=True)
print(df.head(3))

                                             スマートフォン  \
0  \n【セット割】ASUS Zenfone Max M2 ミッドナイトブラック (4GB/64...   
1  \n【セット割】ASUS Zenfone Max Pro M2 ミッドナイトブルー (6GB...   
2  \nスマホ 本体 Blackview BV5900 simフリー スマートフォン 本体 4G...   

                                                エプソン  \
0     \nエプソン プリンター インクジェット複合機 カラリオ EP-883AR レッド(赤)\n   
1  \nインクのチップス エプソン用 MUG-BK 互換インク ブラック 4本セット (マグカッ...   
2     \nエプソン用 ITH(イチョウ)互換インク ITH-6CL 互換 6色+黒1本 計7本\n   

                                               化粧落とし  
0  \nファンケル (FANCL) 新マイルドクレンジング オイル 1本 120mL (約60回...  
1  \nkikimate キキメイト クレンジングジェル メイク落とし クレンジングフォーム【W...  
2  \nDove(ダヴ) ダヴ ボタニカルセレクション ポアビューティー 限定処方 オイルクレン...  


In [4]:
def tokenize(df):

    tokenizer = Tokenizer()
    token_filters = [POSStopFilter(['記号', '助詞', '助動詞'])]
    analysis = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

    corpus = []
    for col in df.columns.to_list():
        df_tmp = df[col].dropna()
        sentence_concat = concat_str(df_tmp.values)
        token = analysis.analyze(sentence_concat)
        word_list = [word.surface for word in token]
        word_concat = concat_str(word_list)
        corpus.append(word_concat)

    return corpus

In [5]:
def concat_str(string_list):

    string_concat = ''
    for string in string_list:
        string_concat = string_concat + string + ' '

    return string_concat

In [8]:
corpus = tokenize(df)
print(corpus[0][0:5])

セット 割


In [9]:
def calc_tfidf(corpus):

    vec = TfidfVectorizer()
    tfidf = vec.fit_transform(corpus).toarray()
    features = vec.get_feature_names()

    return features, tfidf

In [22]:
features, tfidf = calc_tfidf(corpus)
print('features size: ', len(features))
print(features[100:105])
print('tfidf size: ', tfidf.shape)
print(tfidf[0][100:105])


features size:  1276
['3700', '39', '3900', '400', '4000']
tfidf size:  (3, 1276)
[0.         0.00352659 0.00352659 0.         0.01410637]


In [23]:
def sort_result(features, tfidf):

    result_list = []
    for values in tfidf:
        result_dict = dict(zip(features, values))
        result_dict_sort = {
            ky: val
            for ky, val in sorted(result_dict.items(),
                                  key=lambda item: item[1], reverse=True)
        }
        result_list.append(result_dict_sort)

    return result_list

In [24]:
result_list = sort_result(features, tfidf)

In [25]:
print(type(result_list))
for result in result_list:
    print(len(result))

<class 'list'>
1276
1276
1276
